                                        #### Project 4

                                        #### Project 4
# API on College Data. Collecting it. Scraping it... Whatever it takes

_Author: Alex Chicote (NYC)_

---
<img src="./pexels-photo-356086.jpeg" style="float: center; margin: 20px; height: 300px">

                                            Let's get the necessary data from schools and cost in
                                            college tuition in the US?
        

## The main reason I picked this project is because I wanted to get some more practice in web scraping and API. And certainly, I did.
 

## First: import all necessary modules

In [1]:
import pandas as pd
import requests
import json
import pickle
import numpy as np

## Second: Study the possibilities of the data. What is there available?  and what do I want to download? My only clear idea beforehand is to work with API

1. Obtaining the API Key
2. Download/upload the data dictionary
3. I pull out three lists: One conatining the fields related to the root values, a second one for each school a third one for cost

In [2]:
data_dict = pd.read_excel('Workbook1.xlsx')

In [3]:
data_dict[data_dict['dev-category'] == 'root']

,NAME OF DATA ELEMENT,dev-category,developer-friendly name,API data type,VARIABLE NAME,VALUE,LABEL,SOURCE,NOTES
0,Unit ID for institution,root,id,integer,UNITID,NaN,NaN,IPEDS,Shown/used on consumer website.
1,8-digit OPE ID for institution,root,ope8_id,integer,OPEID,NaN,NaN,IPEDS,Shown/used on consumer website.
2,6-digit OPE ID for institution,root,ope6_id,integer,OPEID6,NaN,NaN,IPEDS,Shown/used on consumer website.
116,Latitude,root,location.lat,float,LATITUDE,NaN,NaN,IPEDS,NaN
117,Longitude,root,location.lon,float,LONGITUDE,NaN,NaN,IPEDS,NaN


In [4]:
root_list = data_dict['developer-friendly name'][data_dict['dev-category'] == 'root']

In [5]:
school_list = data_dict['developer-friendly name'][(data_dict['dev-category'] == 'school') & (data_dict['developer-friendly name'].notnull())]

In [6]:
cost_list = data_dict['developer-friendly name'][(data_dict['dev-category'] == 'cost') & (data_dict['developer-friendly name'].notnull())]

In [7]:
cost_list_selection = cost_list[0:2]
cost_list_selection

555     avg_net_price.public
556    avg_net_price.private
Name: developer-friendly name, dtype: object

## Right now I am leaning towards making some kind of study to analyzed what are the most important features that determine the cost of a college.

### List of fields per year (2015 to 2011) for cost

In [8]:
new_cost_list = []
for item in cost_list[0:2]:
    nou = 'cost.'+str(item)
    new_cost_list.append(nou)
final_cost_list = ''
for item in new_cost_list:
    for year in [2011,2012,2013,2014,2015]:
        final_cost_list = final_cost_list + ',' +str(year)+ '.' + item
final_cost_list[1:]

'2011.cost.avg_net_price.public,2012.cost.avg_net_price.public,2013.cost.avg_net_price.public,2014.cost.avg_net_price.public,2015.cost.avg_net_price.public,2011.cost.avg_net_price.private,2012.cost.avg_net_price.private,2013.cost.avg_net_price.private,2014.cost.avg_net_price.private,2015.cost.avg_net_price.private'

### List of fields to get from school related fields

In [9]:
##Preparing the list for school related fields
new_school_list = []
for item in school_list:
    nou = 'school.'+str(item)
    new_school_list.append(nou)

final_school_list = ''
for item in new_school_list:
    final_school_list = final_school_list + str(',') + item

final_school_list = final_school_list[1:]
final_school_list = final_school_list + ',' + 'id'
final_school_list

'school.name,school.city,school.state,school.zip,school.accreditor,school.school_url,school.price_calculator_url,school.degrees_awarded.predominant_recoded,school.under_investigation,school.main_campus,school.branches,school.degrees_awarded.predominant,school.degrees_awarded.highest,school.ownership,school.state_fips,school.region_id,school.locale,school.degree_urbanization,school.carnegie_basic,school.carnegie_undergrad,school.carnegie_size_setting,school.minority_serving.historically_black,school.minority_serving.predominantly_black,school.minority_serving.annh,school.minority_serving.tribal,school.minority_serving.aanipi,school.minority_serving.hispanic,school.minority_serving.nant,school.men_only,school.women_only,school.religious_affiliation,school.online_only,school.operating,school.tuition_revenue_per_fte,school.instructional_expenditure_per_fte,school.faculty_salary,school.ft_faculty_rate,school.alias,school.institutional_characteristics.level,school.open_admissions_policy,scho

In [10]:
final_list =final_school_list + final_cost_list
final_list

'school.name,school.city,school.state,school.zip,school.accreditor,school.school_url,school.price_calculator_url,school.degrees_awarded.predominant_recoded,school.under_investigation,school.main_campus,school.branches,school.degrees_awarded.predominant,school.degrees_awarded.highest,school.ownership,school.state_fips,school.region_id,school.locale,school.degree_urbanization,school.carnegie_basic,school.carnegie_undergrad,school.carnegie_size_setting,school.minority_serving.historically_black,school.minority_serving.predominantly_black,school.minority_serving.annh,school.minority_serving.tribal,school.minority_serving.aanipi,school.minority_serving.hispanic,school.minority_serving.nant,school.men_only,school.women_only,school.religious_affiliation,school.online_only,school.operating,school.tuition_revenue_per_fte,school.instructional_expenditure_per_fte,school.faculty_salary,school.ft_faculty_rate,school.alias,school.institutional_characteristics.level,school.open_admissions_policy,scho

## Creating dataframe conatining all data

In [271]:

cost_dict_list = []
school_df = pd.DataFrame()
for page in range(0,76):
    url = 'https://api.data.gov/ed/collegescorecard/v1/schools.json?api_key=ZmsEq2zy6gjh5XNiIhUAlkgpZZ9NgR4mlyP4Fc6N&_fields=fields&_page=page&_per_page=100'
    api_key = 'ZmsEq2zy6gjh5XNiIhUAlkgpZZ9NgR4mlyP4Fc6N'
    #fields = final_cost_list_14
    params = {'_fields': [final_list], '_page': page}    
    res = requests.get(url,params) 
            # Convert the result, which is a string into a python list
    dictionary = json.loads(res.content)
    sc2 = pd.DataFrame(dictionary['results'])
    school_df = school_df.append(sc2)
    school_df.to_csv('school_df.csv', encoding='utf-8')

# This is where I restart if need it

In [11]:
school_df = pd.read_csv('school_df.csv')
school_df.head()

,Unnamed: 0,2011.cost.avg_net_price.private,2011.cost.avg_net_price.public,2012.cost.avg_net_price.private,2012.cost.avg_net_price.public,2013.cost.avg_net_price.private,2013.cost.avg_net_price.public,2014.cost.avg_net_price.private,2014.cost.avg_net_price.public,2015.cost.avg_net_price.private,...,school.region_id,school.religious_affiliation,school.school_url,school.state,school.state_fips,school.title_iv.approval_date,school.tuition_revenue_per_fte,school.under_investigation,school.women_only,school.zip
0,0,13109.0,NaN,13601.0,NaN,13754.0,NaN,15361.0,NaN,16621.0,...,3,NaN,www.nim.edu,OH,39,10/17/2003,5251.0,1,0.0,44319-2100
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,NaN,www.northshore.org/academics/other-programs/sc...,IL,17,9/20/2002,13314.0,0,0.0,60201-1718
2,2,280.0,NaN,3486.0,NaN,16025.0,NaN,14860.0,NaN,13905.0,...,6,88.0,www.familyoffaith.edu,OK,40,9/24/2002,1125.0,0,0.0,74802
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5,24.0,www.hoodseminary.edu,NC,37,7/15/2002,11364.0,0,0.0,28144
4,4,21891.0,NaN,23289.0,NaN,24405.0,NaN,24247.0,NaN,23896.0,...,2,NaN,www.pcage.edu,NJ,34,11/20/2007,6184.0,0,0.0,08830


In [12]:
school_df.shape

(7593, 54)

In [13]:
school_df.drop('Unnamed: 0', axis=1, inplace=True)

In [14]:
#it needs re indexing
school_df['id'][95:105]

95       457077
96       457086
97     10722001
98     10722002
99     11835605
100    11835606
101    11835608
102    11835609
103    18017901
104    18945901
Name: id, dtype: int64

In [15]:
school_df = school_df.reset_index(drop=True)

## I want to make one column for cost(my target). In order to do it, first I will create 2 columns:
### The first one is the average cost for private colleges from 2011 to 2015 ['private_cost]
### The second one is the same but for public colleges['public_cost]

In [16]:
cost_public = [c for c in school_df.columns if '_price.public' in c]
cost_private = [c for c in school_df.columns if '_price.private' in c]
school_df['private_cost'] = school_df[cost_private].mean(axis=1)
school_df['public_cost'] = school_df[cost_public].mean(axis=1)

### It does not have much sense to me a negative price for a private or public school. I will delete them

In [17]:
school_df[school_df['private_cost'] < 0]['private_cost']

2231   -28912.0
5596    -6210.2
7444    -1265.5
Name: private_cost, dtype: float64

In [18]:
school_df[school_df['public_cost'] < 0]['public_cost']

1216   -123.6
4018   -309.0
6185   -547.2
Name: public_cost, dtype: float64

In [19]:
school_df.drop(school_df.index[[2231, 5596, 7444,1216, 4018, 6185]], inplace=True)

In [20]:
school_df[school_df['private_cost'] < 0]['private_cost']

Series([], Name: private_cost, dtype: float64)

In [21]:
school_df[school_df['public_cost'] < 0]['public_cost']

Series([], Name: public_cost, dtype: float64)

In [22]:
school_df.shape

(7587, 55)

### Making sure I am not missing anything

In [23]:
a = school_df.loc[(school_df['private_cost'].notnull())\
                  & (school_df['public_cost'].isnull()), :].count()['school.name']

b = school_df.loc[(school_df['private_cost'].isnull())\
                   & (school_df['public_cost'].notnull()), :].count()['school.name']
print(school_df.shape[0] - a - b)
school_df.loc[(school_df['private_cost'].isnull())\
                   & (school_df['public_cost'].isnull()), :].count()['school.name']


985


983

985 colleges do not match. 983 have no cost data at all.

In [24]:
checking_it_out = school_df.loc[(school_df['private_cost'].isnull())\
                   & (school_df['public_cost'].isnull()), :]
##Doublechecking
checking_it_out.isnull().sum()


2011.cost.avg_net_price.private                983
2011.cost.avg_net_price.public                 983
2012.cost.avg_net_price.private                983
2012.cost.avg_net_price.public                 983
2013.cost.avg_net_price.private                983
2013.cost.avg_net_price.public                 983
2014.cost.avg_net_price.private                983
2014.cost.avg_net_price.public                 983
2015.cost.avg_net_price.private                983
2015.cost.avg_net_price.public                 983
id                                               0
school.accreditor                              148
school.accreditor_code                         148
school.alias                                   777
school.branches                                  0
school.carnegie_basic                          446
school.carnegie_size_setting                   446
school.carnegie_undergrad                      446
school.city                                      0
school.degree_urbanization     

In [25]:
school_df.dropna(subset=['private_cost','public_cost'], how='all', inplace=True)

In [26]:
school_df.shape

(6604, 55)

# Schools that have data as public AND private

In [27]:
school_df.loc[(school_df['private_cost'].notnull())\
                   & (school_df['public_cost'].notnull()), :].count()['school.name']

2

In [28]:
school_df.loc[(school_df['private_cost'].notnull())\
                   & (school_df['public_cost'].notnull()), :] 

,2011.cost.avg_net_price.private,2011.cost.avg_net_price.public,2012.cost.avg_net_price.private,2012.cost.avg_net_price.public,2013.cost.avg_net_price.private,2013.cost.avg_net_price.public,2014.cost.avg_net_price.private,2014.cost.avg_net_price.public,2015.cost.avg_net_price.private,2015.cost.avg_net_price.public,...,school.school_url,school.state,school.state_fips,school.title_iv.approval_date,school.tuition_revenue_per_fte,school.under_investigation,school.women_only,school.zip,private_cost,public_cost
4471,NaN,14823.0,NaN,13845.0,15569.0,NaN,16419.0,NaN,18181.0,NaN,...,paulmitchell.edu/indianapolis/,IN,18,1/13/2011,9534.0,0,0.0,46227-6252,16723.0,14334.0
4552,NaN,NaN,NaN,6636.0,9234.0,NaN,8015.0,NaN,9763.0,NaN,...,www.larrysbarbercollege.com,IL,17,7/27/2011,2690.0,0,0.0,60628-2322,9004.0,6636.0


In [29]:
school_df['private_cost'] = school_df['private_cost'].fillna(0)
school_df['public_cost'] = school_df['public_cost'].fillna(0)
school_df['cost'] = school_df['private_cost'] + school_df['public_cost']

In [30]:
school_df['cost'].isnull().sum()

0

I am going to fix the 2 that had private and public values

In [31]:
school_df.loc[(school_df['private_cost'] > 0)\
                   & (school_df['public_cost'] > 0),:]

,2011.cost.avg_net_price.private,2011.cost.avg_net_price.public,2012.cost.avg_net_price.private,2012.cost.avg_net_price.public,2013.cost.avg_net_price.private,2013.cost.avg_net_price.public,2014.cost.avg_net_price.private,2014.cost.avg_net_price.public,2015.cost.avg_net_price.private,2015.cost.avg_net_price.public,...,school.state,school.state_fips,school.title_iv.approval_date,school.tuition_revenue_per_fte,school.under_investigation,school.women_only,school.zip,private_cost,public_cost,cost
4471,NaN,14823.0,NaN,13845.0,15569.0,NaN,16419.0,NaN,18181.0,NaN,...,IN,18,1/13/2011,9534.0,0,0.0,46227-6252,16723.0,14334.0,31057.0
4552,NaN,NaN,NaN,6636.0,9234.0,NaN,8015.0,NaN,9763.0,NaN,...,IL,17,7/27/2011,2690.0,0,0.0,60628-2322,9004.0,6636.0,15640.0


In [32]:
school_df.loc[4471,'cost'] = 31057/2
school_df.loc[4552, 'cost'] = 15640/2

In [33]:
school_df['cost'].describe()

count     6604.000000
mean     15514.175972
std       7586.988207
min        151.400000
25%       9386.750000
50%      15093.400000
75%      20544.550000
max      89439.500000
Name: cost, dtype: float64

## Dropping colmuns
### I will drop the yearly cost columns, except for cost. Although private cost and public cost might be useful, I can only recover them through ownership.

In [34]:
school_df.shape

(6604, 56)

In [35]:
school_df.drop(cost_private, axis=1, inplace=True)
school_df.drop(cost_public, axis=1, inplace=True)#cost_private

In [36]:
school_df.drop(['private_cost', 'public_cost'], axis=1, inplace=True)

In [37]:
##Getting rid of school thing in columns
#new_columns = [c[7:] for c in school_df.columns if c not in do_not_touch else c]
school_df.columns = [c[7:] if c.startswith('school') else c for c in school_df.columns]

In [38]:
school_df.columns

Index(['id', 'accreditor', 'accreditor_code', 'alias', 'branches',
       'carnegie_basic', 'carnegie_size_setting', 'carnegie_undergrad', 'city',
       'degree_urbanization', 'degrees_awarded.highest',
       'degrees_awarded.predominant', 'degrees_awarded.predominant_recoded',
       'faculty_salary', 'ft_faculty_rate',
       'institutional_characteristics.level',
       'instructional_expenditure_per_fte', 'locale', 'main_campus',
       'men_only', 'minority_serving.aanipi', 'minority_serving.annh',
       'minority_serving.hispanic', 'minority_serving.historically_black',
       'minority_serving.nant', 'minority_serving.predominantly_black',
       'minority_serving.tribal', 'name', 'online_only',
       'open_admissions_policy', 'operating', 'ownership',
       'price_calculator_url', 'region_id', 'religious_affiliation',
       'school_url', 'state', 'state_fips', 'title_iv.approval_date',
       'tuition_revenue_per_fte', 'under_investigation', 'women_only', 'zip',
       'c

## Analyzing nulls in columns

In [39]:
[school_df.isnull().sum()]

[id                                         0
 accreditor                               661
 accreditor_code                          661
 alias                                   4414
 branches                                   0
 carnegie_basic                             0
 carnegie_size_setting                      0
 carnegie_undergrad                         0
 city                                       0
 degree_urbanization                     6604
 degrees_awarded.highest                    0
 degrees_awarded.predominant                0
 degrees_awarded.predominant_recoded     6604
 faculty_salary                          2523
 ft_faculty_rate                         2981
 institutional_characteristics.level        0
 instructional_expenditure_per_fte          8
 locale                                     0
 main_campus                                0
 men_only                                   0
 minority_serving.aanipi                    0
 minority_serving.annh            

## degrees_awarded.predominant_recoded and degree_urbanization

### All nulls: i will drop them

In [40]:
school_df.drop(['degrees_awarded.predominant_recoded', 'degree_urbanization'], axis=1,inplace=True)

## Religious affiliation
### Must be an integer from -2 (not appliable), -1(not reported), different values from 22 to 103 for different affiliations

In [41]:
data_dict[(data_dict['dev-category'] == 'school') & (data_dict['developer-friendly name'] == 'religious_affiliation')]

,NAME OF DATA ELEMENT,dev-category,developer-friendly name,API data type,VARIABLE NAME,VALUE,LABEL,SOURCE,NOTES
208,Religous affiliation of the institution,school,religious_affiliation,integer,RELAFFIL,-1.0,Not reported,IPEDS,Shown/used on consumer website.


In [42]:
school_df[(school_df['religious_affiliation']>=22) & (school_df['religious_affiliation'] <=103)].count()['cost']

768

## It is my understanding that 768 values for affiliation in the dataset and the difference 5842 must be considered -1 or -2. I will assign value 1 to those that have affiliation and zero to the nulls

In [43]:
school_df['religious_affiliation'] = [1 if c >= 22 else 0 for c in school_df['religious_affiliation']]

In [44]:
school_df['religious_affiliation'].value_counts()

0    5835
1     769
Name: religious_affiliation, dtype: int64

### Same thing: if they have alias 1 if not 0 that corresponds with nulls

In [46]:
school_df['alias'] = school_df['alias'].fillna(0)
school_df['alias'] = [c if c == 0 else 1 for c in school_df['alias']]

In [47]:
school_df['alias'].value_counts()

0    4414
1    2190
Name: alias, dtype: int64

## Faculty_salary and faculty_rate
### Salary per month for full time employees (faculty_salary)
### average of full time employees over total (ft_salry_rate) 0-1
### I will fill them up with the mean. This way I will also have the info about who they are

In [48]:

#Whatever it is it will not be useful for me
school_df[['ft_faculty_rate', 'faculty_salary']].describe()

,ft_faculty_rate,faculty_salary
count,3623.000000,4081.000000
mean,0.564079,6235.119579
std,0.300233,2270.812471
min,0.000000,361.000000
25%,0.306850,4718.000000
50%,0.533300,6015.000000
75%,0.845450,7533.000000
max,1.000000,21354.000000


In [49]:
salary_mean = school_df['faculty_salary'].mean()
faculty_rate_mean = school_df['ft_faculty_rate'].mean()
school_df['ft_faculty_rate'] = school_df['ft_faculty_rate'].fillna(faculty_rate_mean)
school_df['faculty_salary'] = school_df['faculty_salary'].fillna(salary_mean)

## Accreditor and accreditor_code
### Accreditor and its code. it has sense that when one is null the other one too
### I will keep [accreditor] for now to eliminate it later with a bunch of others

In [50]:
school_df[['accreditor_code', 'accreditor']].head()

,accreditor_code,accreditor
0,NaN,NaN
2,ABHE,Association for Bibical Higher Educaiton
4,ACICS,Accrediting Council for Independent Colleges a...
5,ACCSC,Accrediting Commission of Career Schools and C...
6,ACCSC,Accrediting Commission of Career Schools and C...


In [51]:
###I will added a new accreditor code AAAA that will stand for those who have none or it is not available
school_df['accreditor_code'].unique()
school_df['accreditor_code'] = school_df['accreditor_code'].fillna('AAAA')

## Open admissions policy
### 1 is for yes, 2 is for no and 3 for no first-time students
### I am going to assume that nulls belong to 3 as there are no 3 in the dataset. It is also no too much relevant being only 65 of them

In [52]:
school_df['open_admissions_policy'] = school_df['open_admissions_policy'].fillna(3)

## Last ones
### school_url: I will drop column (with accreditor)
### instructional_expenditure_per_fte, tuition_revenue_per_fte and titleiv... I will fill na with mean

In [53]:
ultims = ['school_url', 'tuition_revenue_per_fte', 'title_iv.approval_date']

In [54]:
school_df.drop(['school_url', 'accreditor'], axis=1, inplace=True)

In [55]:
mean_fte = school_df['tuition_revenue_per_fte'].mean()
school_df['tuition_revenue_per_fte'] = school_df['tuition_revenue_per_fte'].fillna(mean_fte)

In [56]:
school_df['title_iv.approval_date'].dtypes
school_df['title_iv.approval_date'] = pd.to_datetime(school_df['title_iv.approval_date'])

In [57]:
from datetime import date, datetime, timedelta
print(school_df['title_iv.approval_date'].max())
print(school_df['title_iv.approval_date'].min())

2015-11-09 00:00:00
1965-01-01 00:00:00


In [58]:
first_day = datetime(1965, 1, 1)
final_day = datetime(2015, 11, 9)
diff = final_day - first_day
mean_day = first_day + diff/2
mean_day

datetime.datetime(1990, 6, 6, 0, 0)

In [59]:
school_df['title_iv.approval_date'] = school_df['title_iv.approval_date'].fillna(mean_day)

In [60]:
base_date = datetime(1965, 1, 1)
school_df['seniority'] = school_df['title_iv.approval_date']- base_date
#school_df['seniority'] = (school_df['seniority'].astype(int))
school_df['seniority'] = school_df['seniority'].dt.days

In [61]:
expend_mean = school_df['instructional_expenditure_per_fte'].mean()
school_df['instructional_expenditure_per_fte'] = school_df['instructional_expenditure_per_fte'].fillna(expend_mean)

In [62]:
school_df.isnull().sum().sum()

0

# Separating columns by dtypes: Object columns and numerical columns

In [63]:
school_df.shape

(6604, 41)

In [64]:
ob_columns = school_df.select_dtypes(include=['object']).columns
len(ob_columns)

6

In [65]:
num_columns = school_df.select_dtypes(exclude=['object']).columns
len(num_columns)

35

# Of all object columns I will only use accreditor_code. I will get dummies out of it
## The other ones give an information already contained in the numeric columns:
## state_fips and region-id play the role of zip and state
## id is as good as name
## price calculator url I consider it irrelevant

In [66]:
school_df[ob_columns].head()

,accreditor_code,city,name,price_calculator_url,state,zip
0,AAAA,Akron,National Institute of Massotherapy,www.nim.edu/NetPrice/npcalc.htm,OH,44319-2100
2,ABHE,Shawnee,Family of Faith Christian University,familyoffaith.edu/netprice/npcalc.htm,OK,74802
4,ACICS,Iselin,PC AGE-Metropark,pcage.edu/calculator/npcalc.htm,NJ,08830
5,ACCSC,Miami,South Florida Institute of Technology,www.sf-institute.com/netprice/npcalc.htm,FL,33125
6,ACCSC,North Charleston,Southeastern Institute-Charleston,www.southeasterninstitute.edu/financial_aid.html,SC,29406


In [67]:
school_df[num_columns].dtypes

id                                               int64
alias                                            int64
branches                                         int64
carnegie_basic                                 float64
carnegie_size_setting                          float64
carnegie_undergrad                             float64
degrees_awarded.highest                          int64
degrees_awarded.predominant                      int64
faculty_salary                                 float64
ft_faculty_rate                                float64
institutional_characteristics.level              int64
instructional_expenditure_per_fte              float64
locale                                         float64
main_campus                                      int64
men_only                                       float64
minority_serving.aanipi                        float64
minority_serving.annh                          float64
minority_serving.hispanic                      float64
minority_s

In [68]:
school_df.loc[school_df['locale'] == -3, ['locale']] = 51
##This weird values corresponds to the marsahall islands and palau, etc.
##As the code of locale goes from smaller(big city to bigger more assolation
## I will assign them the next one 51

In [69]:
len(school_df['accreditor_code'].unique())

35

In [70]:
print((pd.get_dummies(school_df['accreditor_code'])).shape)
print(school_df[num_columns].shape)

(6604, 35)
(6604, 35)


In [104]:
final_dataframe = pd.DataFrame()

In [105]:
final_dataframe = pd.concat([school_df[num_columns], pd.get_dummies(school_df['accreditor_code'])], axis=1)

## Finally, I will erase columns that are directly related to the target.
    ### Tuition revenue for full time employee

In [106]:
final_dataframe.drop('tuition_revenue_per_fte', axis=1, inplace=True)

In [107]:
num_columns

Index(['id', 'alias', 'branches', 'carnegie_basic', 'carnegie_size_setting',
       'carnegie_undergrad', 'degrees_awarded.highest',
       'degrees_awarded.predominant', 'faculty_salary', 'ft_faculty_rate',
       'institutional_characteristics.level',
       'instructional_expenditure_per_fte', 'locale', 'main_campus',
       'men_only', 'minority_serving.aanipi', 'minority_serving.annh',
       'minority_serving.hispanic', 'minority_serving.historically_black',
       'minority_serving.nant', 'minority_serving.predominantly_black',
       'minority_serving.tribal', 'online_only', 'open_admissions_policy',
       'operating', 'ownership', 'region_id', 'religious_affiliation',
       'state_fips', 'title_iv.approval_date', 'tuition_revenue_per_fte',
       'under_investigation', 'women_only', 'cost', 'seniority'],
      dtype='object')

In [108]:
list_of_dummies = ['carnegie_basic', 'locale', 'region_id', 'state_fips']
z = 0
for item in list_of_dummies:
    x = len(school_df[item].unique())
    z = z + x
print(z)
print(final_dataframe.shape)

116
(6604, 69)


In [109]:
for item in list_of_dummies:
    nova = pd.get_dummies(school_df[item], prefix=item)
    final_dataframe = pd.concat([final_dataframe, nova], axis=1)
print(final_dataframe.shape)

(6604, 185)


In [110]:
final_dataframe.to_csv('df_final.csv', index=False, encoding='utf-8')

In [100]:
for item in list_of_dummies:
    print(school_df[item].unique())

[ 10.  24.  -3.  26.  23.  19.  14.   2.  20.  30.  13.  22.  29.  21.  18.
   7.  28.   9.  33.   4.  11.   5.   1.  25.  15.   3.  17.   6.  16.  12.
  32.   8.  31.  27.]
[ 21.  42.  11.  13.  22.  12.  41.  23.  31.  33.  32.  43.  51.]
[3 6 2 5 8 1 4 9 7 0]
[39 40 34 12 45  6 47  9 44 25 42 27 72 35 41 48 36 24  1 51  8 17 26 37 50
  4 31  5 18 21 29 54 49 46 23 22 13 53 20 32 55 16 11 30 28  2 33 15 19 56
 10 38 78 64 66 69 70 60 68]
